In [112]:
import numpy
from scipy.stats import norm
import bs4
import requests
import pandas
import datetime
from bs4 import BeautifulSoup
from datetime import date

today = date.today()


# We need to scrap real-world data first

# define the URL
url = 'https://finance.yahoo.com/quote/FB/options?p=FB'

# open the URL
headers = {"User-Agent":"Mozilla/5.0"}
soup = BeautifulSoup(requests.get(url, headers=headers).content, 'html.parser')

# find the latest option contract
option = soup.find(("tbody",{"data-reactid":"89"}))

num = option.tr.td.a.text # contract number

type = num[8] # determine whether it is a call or put
K_txt = option.tr.td.next_sibling.next_sibling.a.text # determine the strike price of the stock
K = float(K_txt) # determine the strike price of the stock

exp_date = datetime.date(int("20"+num[2:4]),int(num[4:6]),int(num[6:8]))
days = (exp_date-today).days



S = float(option.tr.td.next_sibling.next_sibling.next_sibling.text)# current price
r = 0.01
T = days/365
print(T)
print(days)

sigma_str = option.tr.td.next_sibling.next_sibling.next_sibling.next_sibling.next_sibling.next_sibling.next_sibling.next_sibling.next_sibling.next_sibling.text

sigma = float(sigma_str[:-1])/100
print(sigma)

def blackScholes(r,S,K,T,sigma,type = ""):
    d1 = (numpy.log(S/K) + (r+sigma**2/2)*T)/(sigma*numpy.sqrt(T))
    d2 = d1 - sigma*numpy.sqrt(T)
    try:
        if type =="C":
            price = S*norm.cdf(d1,0,1) - K*numpy.exp(-r*T)*norm.cdf(d2,0,1)
        elif type =="P":
            price = K*numpy.exp(-r*T)*norm.cdf(-d2,0,1) - S*norm.cdf(-d1,0,1)
        return price
    except:
        print('Please double check it is a call or a put')
    
print(blackScholes(r,S,K,T,sigma,type))



0.010958904109589041
4
2.5606
10.851509045279904
